# EdgeQuery 模块测试

**CPRN Edge Query Module Test**

`author` : seika<seika@live.ca>  
`create_date` : `2025-11-24`  
`version` : `v4.1.7.1`  

---

测试新开发的 `EdgeCodeQuery` 模块，验证从数据库查询 edge 记录的功能。

**测试目标**:
- 测试 SQLite 数据库连接
- 测试非嵌套 edge_code 列表查询
- 验证查询结果格式
- 测试转换为 GeoDataFrame


In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# SETTINGS
PATH_SQLITE_EXT: str = r'/opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib'
PATH_SQLITE_CPRN: str = r'/Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite'

# Table name for V417
VERSION_VENDOR = 'V417'
LAYER_NAME_CPRN_SHORT_EDGES_GEOM = f'CPRN_dg_short_edges_geom_{VERSION_VENDOR}_jiangsu'

print(f"📂 Database: {PATH_SQLITE_CPRN}")
print(f"📋 Table: {LAYER_NAME_CPRN_SHORT_EDGES_GEOM}")


📂 Database: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
📋 Table: CPRN_dg_short_edges_geom_V417_jiangsu


In [2]:
# Import EdgeQuery module
from cprn.model.edge_query import EdgeCodeQuery
import geopandas as gpd
import pandas as pd

# Test edge codes
edge_codes_test = [
    'WTTF9GRJER7F001334_WTTF9F0XWRW7001574',
    'WTTF9GRNRN0D001332_WTTF9GRJER7F001334',
    'WTTF9GX9JNU4001323_WTTF9GRNRN0D001332',
    'WTTFD58XYQZN001295_WTTF9GX9JNU4001323']

print(f"🔍 Test edge codes ({len(edge_codes_test)}):")
for i, code in enumerate(edge_codes_test, 1):
    print(f"  {i}. {code}")


🔍 Test edge codes (4):
  1. WTTF9GRJER7F001334_WTTF9F0XWRW7001574
  2. WTTF9GRNRN0D001332_WTTF9GRJER7F001334
  3. WTTF9GX9JNU4001323_WTTF9GRNRN0D001332
  4. WTTFD58XYQZN001295_WTTF9GX9JNU4001323


In [4]:
# Initialize EdgeCodeQuery with SQLite
print("🚀 Initializing EdgeCodeQuery...")
query = EdgeCodeQuery(
    db_config=PATH_SQLITE_CPRN,
    table_name=LAYER_NAME_CPRN_SHORT_EDGES_GEOM,
    spatialite_ext_path=PATH_SQLITE_EXT,
    verbose=True)

print("✅ EdgeCodeQuery initialized successfully!")

2025-11-30 15:30:24.514 | INFO     | cprn.model.edge_query:__init__:98 - Initialized EdgeCodeQuery with SQLite: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite, table: CPRN_dg_short_edges_geom_V417_jiangsu


🚀 Initializing EdgeCodeQuery...
✅ EdgeCodeQuery initialized successfully!


In [5]:
# Query edge records
print("🔍 Querying edge records...")
result = query.query(edge_codes_test)

print(f"\n📊 Query result type: {type(result)}")
print(f"📊 Number of records: {len(result)}")

if result:
    print(f"\n📋 First record keys: {list(result[0].keys())}")
    print(f"\n📝 First record sample:")
    first_record = result[0]
    for key, value in list(first_record.items())[:10]:  # Show first 10 fields
        if key != 'geom':  # Skip geometry for now
            print(f"  {key}: {value}")
    if 'geom' in first_record:
        print(f"  geom: {type(first_record['geom'])} - {first_record['geom'].__class__.__name__}")


2025-11-30 15:30:29.743 | DEBUG    | cprn.model.edge_query:_query_single_sqlite:265 - SQLite SQL: 
            SELECT ogc_fid, edge_code, edge_id, src_vtx, tgt_vtx, rcode, mdir, weight, cls, knd, rtype, lane, con_type, AsWKT(GEOMETRY, 16) AS geom
            FROM CPRN_dg_short_edges_geom_V417_jiangsu
            WHERE edge_code IN ('WTTF9GRJER7F001334_WTTF9F0XWRW7001574', 'WTTF9GRNRN0D001332_WTTF9GRJER7F001334', 'WTTF9GX9JNU4001323_WTTF9GRNRN0D001332', 'WTTFD58XYQZN001295_WTTF9GX9JNU4001323')
            
2025-11-30 15:30:29.743 | INFO     | taisl_sop.data.base.spatialite:__init__:42 - SpatialitePuller initiated with path_sqlite: /Users/seika/sync/lab_works/ZSTORE/geodata/spatialite/CPRN2505.sqlite
2025-11-30 15:30:29.743 | INFO     | taisl_sop.data.base.spatialite:__init__:43 - Path_spatialite_ext: /opt/homebrew/Cellar/libspatialite/5.1.0/lib/mod_spatialite.dylib
2025-11-30 15:30:29.744 | INFO     | taisl_sop.data.base.spatialite:__init__:47 - Set_precision: ❌


🔍 Querying edge records...


2025-11-30 15:30:30.076 | INFO     | taisl_sop.data.geometry.handler:_handle_single_geometry:76 - Processed geometry column: geom
2025-11-30 15:30:30.078 | INFO     | cprn.model.edge_query:query:346 - Queried 4 edge codes, got 4 records


Memory usage of df puller : gdf 0.0 MB, df None MB

📊 Query result type: <class 'list'>
📊 Number of records: 4

📋 First record keys: ['ogc_fid', 'edge_code', 'edge_id', 'src_vtx', 'tgt_vtx', 'rcode', 'mdir', 'weight', 'cls', 'knd', 'rtype', 'lane', 'con_type', 'geom']

📝 First record sample:
  ogc_fid: 16280
  edge_code: WTTF9GRJER7F001334_WTTF9F0XWRW7001574
  edge_id: [46870, 70967, 70968, 70963, 70972, 70970, 70971, 70969, 70964, 70965]
  src_vtx: WTTF9GRJER7F001334
  tgt_vtx: WTTF9F0XWRW7001574
  rcode: G1522
  mdir: FWD
  weight: 1105.71
  cls: MR
  knd: FW
  geom: <class 'shapely.geometry.multilinestring.MultiLineString'> - MultiLineString


In [6]:
# Convert to GeoDataFrame
print("🔄 Converting to GeoDataFrame...")
gdf = query.to_geodataframe(result, geometry_column='geom')

print(f"\n📊 GeoDataFrame shape: {gdf.shape}")
print(f"📊 Columns: {list(gdf.columns)}")
print(f"📊 Geometry column: {gdf.geometry.name}")
print(f"📊 CRS: {gdf.crs}")

# Display summary
print("\n📋 DataFrame info:")
print(gdf.info())

print("\n📋 Sample records:")
gdf.head()

2025-11-30 15:30:33.814 | INFO     | cprn.model.edge_query:to_geodataframe:391 - Converted 4 records to GeoDataFrame


🔄 Converting to GeoDataFrame...

📊 GeoDataFrame shape: (4, 14)
📊 Columns: ['ogc_fid', 'edge_code', 'edge_id', 'src_vtx', 'tgt_vtx', 'rcode', 'mdir', 'weight', 'cls', 'knd', 'rtype', 'lane', 'con_type', 'geom']
📊 Geometry column: geom
📊 CRS: None

📋 DataFrame info:
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   ogc_fid    4 non-null      int64   
 1   edge_code  4 non-null      object  
 2   edge_id    4 non-null      object  
 3   src_vtx    4 non-null      object  
 4   tgt_vtx    4 non-null      object  
 5   rcode      4 non-null      object  
 6   mdir       4 non-null      object  
 7   weight     4 non-null      float64 
 8   cls        4 non-null      object  
 9   knd        4 non-null      object  
 10  rtype      4 non-null      object  
 11  lane       4 non-null      float64 
 12  con_type   0 non-null      object  
 13  geom  

,ogc_fid,edge_code,edge_id,src_vtx,tgt_vtx,rcode,mdir,weight,cls,knd,rtype,lane,con_type,geom
0,16280,WTTF9GRJER7F001334_WTTF9F0XWRW7001574,"[46870, 70967, 70968, 70963, 70972, 70970, 709...",WTTF9GRJER7F001334,WTTF9F0XWRW7001574,G1522,FWD,1105.71,MR,FW,NA,3.0,None,"MULTILINESTRING ((120.6726 31.39579, 120.67257..."
1,16281,WTTF9GRNRN0D001332_WTTF9GRJER7F001334,"[45327, 45326, 46869]",WTTF9GRNRN0D001332,WTTF9GRJER7F001334,G1522,FWD,20.13,MR,FW,NA,4.0,None,"MULTILINESTRING ((120.67276 31.39591, 120.6727..."
2,16282,WTTF9GX9JNU4001323_WTTF9GRNRN0D001332,"[139742, 21534, 21535, 28395]",WTTF9GX9JNU4001323,WTTF9GRNRN0D001332,G1522,FWD,78.12,MR,FW,NA,3.0,None,"MULTILINESTRING ((120.67336 31.39639, 120.6733..."
3,16334,WTTFD58XYQZN001295_WTTF9GX9JNU4001323,"[51766, 51689, 51688, 158412, 181776, 181775]",WTTFD58XYQZN001295,WTTF9GX9JNU4001323,G1522,FWD,187.22,MR,FW,NA,4.0,None,"MULTILINESTRING ((120.67479 31.39755, 120.6747..."


In [7]:
# Visualize geometry (if available)
if 'geom' in gdf.columns and len(gdf) > 0:
    print("🗺️ Geometry information:")
    print(f"  Total records: {len(gdf)}")
    print(f"  Geometry type: {gdf.geom_type.unique()}")
    print(f"  Bounds: {gdf.total_bounds}")
    
    # Show basic stats
    if 'weight' in gdf.columns:
        print(f"\n📏 Weight statistics:")
        print(f"  Min: {gdf['weight'].min():.2f}m")
        print(f"  Max: {gdf['weight'].max():.2f}m")
        print(f"  Mean: {gdf['weight'].mean():.2f}m")
        print(f"  Sum: {gdf['weight'].sum():.2f}m")
else:
    print("⚠️ No geometry column found or empty result")

🗺️ Geometry information:
  Total records: 4
  Geometry type: ['MultiLineString']
  Bounds: [120.66380536  31.38926775 120.67479271  31.39754683]

📏 Weight statistics:
  Min: 20.13m
  Max: 1105.71m
  Mean: 347.80m
  Sum: 1391.18m


In [8]:
gdf.set_crs('epsg:4326').explore()

## 验证查询结果

检查查询到的 edge_code 是否与输入匹配


In [8]:
# Verify query results
if 'edge_code' in gdf.columns:
    print("✅ Verification:")
    print(f"  Input edge codes: {len(edge_codes_test)}")
    print(f"  Retrieved edge codes: {len(gdf)}")
    
    retrieved_codes = set(gdf['edge_code'].tolist())
    input_codes = set(edge_codes_test)
    
    matched = retrieved_codes & input_codes
    missing = input_codes - retrieved_codes
    
    print(f"\n  ✅ Matched: {len(matched)}/{len(input_codes)}")
    if matched:
        for code in matched:
            print(f"    ✓ {code}")
    
    if missing:
        print(f"\n  ⚠️ Missing: {len(missing)}")
        for code in missing:
            print(f"    ✗ {code}")
    
    # Show edge code column
    print("\n📋 Edge codes in result:")
    print(gdf[['edge_code', 'src_vtx', 'tgt_vtx']].to_string() if 'src_vtx' in gdf.columns else gdf[['edge_code']].to_string())
else:
    print("⚠️ 'edge_code' column not found in result")


✅ Verification:
  Input edge codes: 4
  Retrieved edge codes: 4

  ✅ Matched: 4/4
    ✓ WTTFD58XYQZN001295_WTTF9GX9JNU4001323
    ✓ WTTF9GRNRN0D001332_WTTF9GRJER7F001334
    ✓ WTTF9GRJER7F001334_WTTF9F0XWRW7001574
    ✓ WTTF9GX9JNU4001323_WTTF9GRNRN0D001332

📋 Edge codes in result:
                               edge_code             src_vtx             tgt_vtx
0  WTTF9GRJER7F001334_WTTF9F0XWRW7001574  WTTF9GRJER7F001334  WTTF9F0XWRW7001574
1  WTTF9GRNRN0D001332_WTTF9GRJER7F001334  WTTF9GRNRN0D001332  WTTF9GRJER7F001334
2  WTTF9GX9JNU4001323_WTTF9GRNRN0D001332  WTTF9GX9JNU4001323  WTTF9GRNRN0D001332
3  WTTFD58XYQZN001295_WTTF9GX9JNU4001323  WTTFD58XYQZN001295  WTTF9GX9JNU4001323


## EASY TEST

In [12]:
edge_codes = ['WTUD22ZZJ5JW002618_WTUD23NXW6BW002780', 'WTUD23NXW6BW002780_WTUD23Q85TFB002789', 'WTUD23Q85TFB002789_WTUD25NQYN1E003827', 
'WTUD25NQYN1E003827_WTUD25SZ23KC003576', 'WTUD25SZ23KC003576_WTUD2H6V0UN9003308', 'WTUD2H6V0UN9003308_WTUD2HD6CW2D003247', 
'WTUD2HD6CW2D003247_WTU6RVXGUTBX003066', 'WTU6RVXGUTBX003066_WTU6RVZ6WXRY003081', 'WTU6RVZ6WXRY003081_WTU6RYNXPQ0Q003011', 
'WTU6RYNXPQ0Q003011_WTU6RZSSGQGK003450', 'WTU6RZSSGQGK003450_WTU6XB7FFCMN003530', 'WTU6XB7FFCMN003530_WTU6XBEDD692003402', 
'WTU6XBEDD692003402_WTU6XBGPKHDK003021', 'WTU6XBGPKHDK003021_WTU6XSQEV4P4003308', 'WTU6XSQEV4P4003308_WTU6XTHXV05V003534', 
'WTU6XTHXV05V003534_WTU6XW45G23J003905', 'WTU6XW45G23J003905_WTU6XRN7Z6VH004096', 'WTU6XRN7Z6VH004096_WTU6Z24QQKQE004113', 
'WTU6Z24QQKQE004113_WTU6Z1NK7338003694', 'WTU6Z1NK7338003694_WTU6Z1KWPTD2003947', 'WTU6Z1KWPTD2003947_WTU6Z1KRVWRG003968', 
'WTU6Z1KRVWRG003968_WTU6Z4116RKW003614', 'WTU6Z4116RKW003614_WTU6Z40KSSGK003512', 'WTU6Z40KSSGK003512_WTU6YKKTTJCT003210', 
'WTU6YKKTTJCT003210_WTU6YKEN3R2J003396', 'WTU6YKEN3R2J003396_WTU6YJRPST20003270', 'WTU6YJRPST20003270_WTU6YNHS8QXK003496']

In [13]:
res = query.query(edge_codes)
gdf = query.to_geodataframe(res, geometry_column='geom')

2025-11-25 14:01:13.269 | DEBUG    | cprn.model.edge_query:_query_single_sqlite:268 - SQLite SQL: 
            SELECT ogc_fid, edge_code, edge_id, src_vtx, tgt_vtx, rcode, mdir, weight, cls, knd, rtype, lane, con_type, AsWKT(GEOMETRY, 16) AS geom
            FROM CPRN_dg_short_edges_geom_V417_jiangsu
            WHERE edge_code IN ('WTUD22ZZJ5JW002618_WTUD23NXW6BW002780', 'WTUD23NXW6BW002780_WTUD23Q85TFB002789', 'WTUD23Q85TFB002789_WTUD25NQYN1E003827', 'WTUD25NQYN1E003827_WTUD25SZ23KC003576', 'WTUD25SZ23KC003576_WTUD2H6V0UN9003308', 'WTUD2H6V0UN9003308_WTUD2HD6CW2D003247', 'WTUD2HD6CW2D003247_WTU6RVXGUTBX003066', 'WTU6RVXGUTBX003066_WTU6RVZ6WXRY003081', 'WTU6RVZ6WXRY003081_WTU6RYNXPQ0Q003011', 'WTU6RYNXPQ0Q003011_WTU6RZSSGQGK003450', 'WTU6RZSSGQGK003450_WTU6XB7FFCMN003530', 'WTU6XB7FFCMN003530_WTU6XBEDD692003402', 'WTU6XBEDD692003402_WTU6XBGPKHDK003021', 'WTU6XBGPKHDK003021_WTU6XSQEV4P4003308', 'WTU6XSQEV4P4003308_WTU6XTHXV05V003534', 'WTU6XTHXV05V003534_WTU6XW45G23J003905', 'WTU6XW45G

Memory usage of df puller : gdf 0.0 MB, df None MB


In [15]:
gdf.set_crs('epsg:4326').explore()